# 525 Team 11 Milestone 1 

In [1]:
import re
import os,sys,inspect
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage
import dask.dataframe as dd
import numpy as np

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

C:\Users\A\miniconda3\envs\525\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [3]:
# get the folders
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir) # this refers to the project root folder
raw_folder = parentdir + "/data/raw/"

processed_folder = parentdir + "/data/processed/"
if not os.path.exists(raw_folder):
    os.makedirs(raw_folder)
    
if not os.path.exists(processed_folder):
    os.makedirs(processed_folder)

combined_file = processed_folder + "combined_data.csv"

files_to_dl = ["data.zip"] # need only this zip file

# avoid re-loading the data if the file already exists locally
force_download = False # set to True to re-download and unzip the file

# M1 - 3. Download the data
<hr>

In [4]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want

In [5]:
%%time
for file in files:
    if file["name"] in files_to_dl:
        if (force_download or not os.path.exists(raw_folder + file["name"])):
            os.makedirs(raw_folder, exist_ok=True) # create the folder if not exists
            urlretrieve(file["download_url"], raw_folder + file["name"])

Wall time: 0 ns


In [6]:
%%time
# extract the zip file
n_files = len(os.listdir(raw_folder))
if (force_download or n_files != 31): # if we must unzip the latest downloaded file or the file was not unzipped
    with zipfile.ZipFile(os.path.join(raw_folder, "data.zip"), 'r') as f:
        f.extractall(raw_folder)

Wall time: 26.2 s


<br>

# M1 - 4. Combine the data
<hr>

## 4.1 Combine the data using `Pandas`

In [7]:
%%time
%memit
import pandas as pd
use_cols = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]
files = glob.glob(raw_folder + '*NSW.csv') # exclude observed_daily_rainfall_SYD

df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                .assign(model=file[max(file.rfind('/'), file.rfind('\\'))+1:file.index("_daily")])
                for file in files)
              )
df.to_csv(combined_file)

peak memory: 149.07 MiB, increment: 0.00 MiB
Wall time: 9min 47s


In [8]:
print("Size of the combined file:", os.path.getsize(combined_file)/(2**30), "GB")

Size of the combined file: 5.618728716857731 GB


In [9]:
%%time
df = pd.read_csv(combined_file)

Wall time: 1min 57s


In [10]:
print(df.shape)

(62467843, 7)


In [11]:
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


<br>

## 4.2 Combine the data using `Dask`

In [12]:
# %%time
# %%memit
# dask_combined_file = processed_folder + "dask_combined_data.csv"
# ddf = dd.read_csv(raw_folder + '*NSW.csv', assume_missing=True, usecols=use_cols, include_path_column=True)
# ddf.to_csv(dask_combined_file, single_file=True)

In [13]:
# print("Size of the combined file:", os.path.getsize(dask_combined_file)/(2**30), "GB")

In [14]:
%%time
ddf = dd.read_csv(combined_file)
ddf.head()

Wall time: 1.63 s


,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


<br>

## 4.3 Observations:

Our team have **downloaded the data** and **executed the data combination script using `Pandas` on 4 laptops**. Below are the detailed results:

OS|CPU|Memory|Runtime|Memory
--|---|------|-------|------
macOS BigSur|2.7 GHz Dual-Core Intel Core i5|16 GB|CPU times: user 7min 37s, sys: 24.5 s, total: 8min 1s. Wall time: 8min 35s|peak memory: 155.76 MiB, increment: 0.04 MiB
Windows 10 Education Insider Preview|Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz 2.30 GHz|16 GB|Wall time: 8min 55s|peak memory: 138.60 MiB, increment: 0.25 MiB
macOS Catalina version 10.15.7|2.4 GHz Quad-Core Intel Core i5|8 GB|CPU times: user 5min 45s, sys: 21.5 s, total: 6min 7s. Wall time: 6min 15s|peak memory: 125.83 MiB, increment: 0.25 MiB
Windows 10 Education|Intel(R) Core(TM) i7-8550U CPU @ 1.80 GHz 1.99 GHz| 16 GB|Wall time: 10min 11s|peak memory: 140.70 MiB, increment: 0.20 MiB

As we can see from the result table above, the **runtime varied across machines** depending on their configurations, but they **all took considerable amount of time and memory**. It is noteworthy that:

* Due to the limited hard drive space, we had to comment out the script used for **combining data** files with `Dask`; also in the previous execution, using `Dask` to **combine these files actually took more time** than using `Pandas`. 

* `Dask`'s `read_csv` function is, however, super **fast** in comparison to using `Pandas`'s due to the fact that `Pandas` **loads the whole data object into memory** whilst `Dask` **loads data in chunks** and applies parallel processing.

**In conclusion, merging csv files into one giant file and loading it every time is clearly not an efficient way to work with big data files.**

<br>

# M1 - 5. Load the combined CSV to memory and perform a simple EDA in `Python`
<hr>

## 5.1 Changing `dtype` of the data

In [6]:
# Loading the combined dataframe to the memory
df = pd.read_csv("../data/processed/combined_data.csv")
df.head()

,time,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-36.25,-35.0,140.625,142.5,3.293256e-13,ACCESS-CM2
1,1889-01-02 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
2,1889-01-03 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
3,1889-01-04 12:00:00,-36.25,-35.0,140.625,142.5,0.000000e+00,ACCESS-CM2
4,1889-01-05 12:00:00,-36.25,-35.0,140.625,142.5,1.047658e-02,ACCESS-CM2


In [4]:
# Checking data types of the columns
df.dtypes

time              object
lat_min          float64
lat_max          float64
lon_min          float64
lon_max          float64
rain (mm/day)    float64
model             object
dtype: object

In [5]:
#Checking if the max and min values of the float64-type columns are within the range of `float32`
df_numeric_summary = df.describe().loc[["min", "max"], :].T
df_numeric_summary["within float32 range"] = df_numeric_summary[["min", "max"]].apply(lambda x:
                                                                                      "True" if (x["min"] > np.finfo(np.float32).min
                                                                                                 and x["max"] < np.finfo(np.float32).max)
                                                                                             else "False", axis=1)
df_numeric_summary

,min,max,within float32 range
lat_min,-3.646739e+01,-29.900000,True
lat_max,-3.600000e+01,-27.906064,True
lon_min,1.406250e+02,153.750000,True
lon_max,1.412500e+02,155.625000,True
rain (mm/day),-3.807373e-12,432.939515,True


In [6]:
print(f"Memory usage with float64 (default): {df[['lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with float32: {df[['lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)']].astype('float32', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with float64 (default): 2498.71 MB
Memory usage with float32: 1249.36 MB


In [7]:
print(f"Memory usage with object (default) for the time column: {df[['time']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with datetime64 for the time column: {df[['time']].astype('datetime64[ns]', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with object (default) for the time column: 499.74 MB
Memory usage with datetime64 for the time column: 499.74 MB


In [8]:
# Checking the `model` column's unique values
df.model.unique()

array(['MPI-ESM-1-2-HAM', 'AWI-ESM-1-1-LR', 'NorESM2-LM', 'ACCESS-CM2',
       'FGOALS-f3-L', 'CMCC-CM2-HR4', 'MRI-ESM2-0', 'GFDL-CM4',
       'BCC-CSM2-MR', 'EC-Earth3-Veg-LR', 'CMCC-ESM2', 'NESM3',
       'MPI-ESM1-2-LR', 'ACCESS-ESM1-5', 'FGOALS-g3', 'INM-CM4-8',
       'MPI-ESM1-2-HR', 'TaiESM1', 'NorESM2-MM', 'CMCC-CM2-SR5',
       'KIOST-ESM', 'INM-CM5-0', 'MIROC6', 'BCC-ESM1', 'GFDL-ESM4',
       'CanESM5', 'SAM0-UNICON'], dtype=object)

In [9]:
print(f"Memory usage with object (default) for the model column: {df[['model']].memory_usage().sum() / 1e6:.2f} MB")
print(f"Memory usage with string for the model column: {df[['model']].astype('string', errors='ignore').memory_usage().sum() / 1e6:.2f} MB")

Memory usage with object (default) for the model column: 499.74 MB
Memory usage with string for the model column: 499.74 MB


In [10]:
# Calculating memory savings for each column
df_memory_savings = pd.DataFrame(columns = df.columns).T.reset_index().rename(columns={'index': 'column name'})

list_savings = []
savings = 0
for col in df.columns:
    if df[col].dtypes == "float64":
        savings = (df[[col]].memory_usage().sum() / 1e6) - (df[[col]].astype('float32', errors='ignore').memory_usage().sum() / 1e6)
        list_savings.append(savings)
        savings = 0
    elif col == "time":
        savings = (df[["time"]].memory_usage().sum() / 1e6) - (df[[col]].astype('datetime64[ns]', errors='ignore').memory_usage().sum() / 1e6)
        list_savings.append(savings)
        savings = 0
    else:
        savings = (df[["model"]].memory_usage().sum() / 1e6) - (df[[col]].astype('string', errors='ignore').memory_usage().sum() / 1e6)
        list_savings.append(savings)
        savings = 0

df_memory_savings['memory savings'] = pd.Series(list_savings).values
df_memory_savings

,column name,memory savings
0,time,0.000000
1,lat_min,249.871372
2,lat_max,249.871372
3,lon_min,249.871372
4,lon_max,249.871372
5,rain (mm/day),249.871372
6,model,0.000000


In [11]:
print("\033[1m" + f"Memory savings due to changing the data types of the columns: {df_memory_savings['memory savings'].sum():.2f} MB")

Memory savings due to changing the data types of the columns: 1249.36 MB


<br>

## 5.2 Loading just the columns we want

In [12]:
print(f"Memory usage with all columns: {df[['time', 'lat_min','lat_max','lon_min', 'lon_max', 'rain (mm/day)', 'model']].memory_usage().sum() / 1e6:.2f} MB")

Memory usage with all columns: 3498.20 MB


In [13]:
# Dropping `lat_min` and `lon_min` columns
df_reduced = df.drop(['lat_min', 'lon_min'], axis=1);
print(f"Memory usage of the reduced dataframe: {df_reduced.memory_usage().sum() / 1e6:.2f} MB")
df_reduced.head()

Memory usage of the reduced dataframe: 2498.71 MB


,time,lat_max,lon_max,rain (mm/day),model
0,1889-01-01 12:00:00,-33.574619,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1,1889-01-02 12:00:00,-33.574619,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
2,1889-01-03 12:00:00,-33.574619,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
3,1889-01-04 12:00:00,-33.574619,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
4,1889-01-05 12:00:00,-33.574619,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


In [14]:
print("\033[1m" + f"Memory savings due to loading not all columns: {df[['lat_min', 'lon_min']].memory_usage().sum() / 1e6:.2f} MB")

Memory savings due to loading not all columns: 999.49 MB


<br>

## 5.3 Loading data in chunks

In [31]:
%%memit
df = pd.read_csv("../data/processed/combined_data.csv")
df['model'].value_counts()

peak memory: 7295.00 MiB, increment: 5542.13 MiB


In [32]:
print(f"Memory usage with loading all data of the 'model' column and perform value_counts EDA: {7295 * 1.048576:.2f} MB")

Memory usage with loading all data of the 'model' column and perform value_counts EDA: 7649.36 MB


In [29]:
%%memit
values = pd.Series(dtype=object)
for chunk in pd.read_csv("../data/processed/combined_data.csv", chunksize=10_000_000):
    values = values.add(chunk['model'].value_counts(), fill_value=0)
print(values.astype(object))

ACCESS-CM2          1932840.0
ACCESS-ESM1-5       1610700.0
AWI-ESM-1-1-LR       966420.0
BCC-CSM2-MR         3035340.0
BCC-ESM1             551880.0
CMCC-CM2-HR4        3541230.0
CMCC-CM2-SR5        3541230.0
CMCC-ESM2           3541230.0
CanESM5              551880.0
EC-Earth3-Veg-LR    3037320.0
FGOALS-f3-L         3219300.0
FGOALS-g3           1287720.0
GFDL-CM4            3219300.0
GFDL-ESM4           3219300.0
INM-CM4-8           1609650.0
INM-CM5-0           1609650.0
KIOST-ESM           1287720.0
MIROC6              2070900.0
MPI-ESM-1-2-HAM      966420.0
MPI-ESM1-2-HR       5154240.0
MPI-ESM1-2-LR        966420.0
MRI-ESM2-0          3037320.0
NESM3                966420.0
NorESM2-LM           919800.0
NorESM2-MM          3541230.0
SAM0-UNICON         3541153.0
TaiESM1             3541230.0
dtype: object
peak memory: 3699.65 MiB, increment: 2144.54 MiB


In [37]:
print(f"Memory usage with loading in chunks of the 'model' column and perform value_counts EDA: {3699.65 * 1.048576:.2f} MB")

Memory usage with loading in chunks of the 'model' column and perform value_counts EDA: 3879.36 MB


In [36]:
print("\033[1m" + f"Memory savings due to loading in chunks: {(7295-3699.65) * 1.048576:.2f} MB")

Memory savings due to loading in chunks: 3770.00 MB


<br>

## 5.4 `Python` Observations

Our team decided to try 3 approaches to reduce memory usage when performing the EDA in `Python`:

### **Approach 1. Change the data types of the columns in the dataframe.**

There are 2 columns of the `object` type (`time` and `model`) and 5 columns of the `float64` type (`lat_min`, `lat_max`, `lon_min`, `lon_max`, and `rain (mm/day)`).

* The summary table for the numeric columns shows that they are within the range of the `float32` data type, i.e. the numeric columns' highest values do not exceed the `float32` maximum number, and their lowest values are not less than the `float32` minimum. 

* Changing the data type for these columns into the `float32` type **leads to the two-fold decrease in memory usage**, with the **reduction from 2498.71 MB to 1249.36 MB**. 

* It is related to the fact that `float64` allocates twice as much memory as `float32`, since `float64` can store much larger numbers than `float32`.

Regarding the `time` and `model` columns, 499.74 MB of memory was allocated for each of the columns. Interestingly, changing the data type for the `time` and `model` columns to `datetime64[ns]` and `string` respectively did not change the amount of memory used.

Based on the above, we can **conclude that the `datetime64`, `string`, and `object` data types allocate the same amount of memory for storing data**. For this reason, **in order to reduce memory usage it is reasonable to change the data type for numeric columns**.

> In this particular case, **memory savings** due to changing the data types of the numeric columns are **1249.36 MB**.


<hr>

### **Approach 2. Load the specific columns we are interested in.**

For the sake of the task, we suppose that now we are not interested in the `lat_min` and `lon_min` columns, i.e. these columns do not play a big role in our prediction problem. 

* **Dropping the specified columns reduced the memory usage** from **3498.20 MB** to **2498.71 MB**.

This shows the efficiency of loading only those columns that are necessary for a specific task in terms of memory usage when working with big data.

> Here, **memory savings** due to loading not all columns (by dropping two columns) are **999.49 MB**.

<hr>

### **Approach 3. Load in data in chunks.**

For the sake of this task, we assume that now we are only interested in the `model` column and we performed `value_counts` EDA on the `model` column. 

* **Loading data** from the `model` column **in chunks reduced the memory usage** from **7649.36 MB** to **3879.36 MB**.

* This shows the efficiency of loading data in chunk in terms of memory usage when working with big data.

> Here, **memory savings** due to loading data in chunks are **3770.00 MB**.

<br>

# M1 - 6. Perform a simple EDA in `R`
<hr>

In [3]:
import pyarrow.feather as feather

In [5]:
processed_folder = os.path.join(parentdir,"data", "processed")

input_path = os.path.join(processed_folder, "combined_data.csv")
output_path = os.path.join(processed_folder, "combined_data.feather")

In [6]:
# %%time
# %memit

# df = pd.read_csv(input_path)

In [7]:
# %%time
# %memit

# feather.write_feather(df, output_path)

In [8]:
# %%R

# install.packages("dplyr")
# library(arrow)

## 6.1 Use `Feather` to transfer from `Python` to `R`

In [3]:
%%time
%%R
library(dplyr)
library(arrow)
start_time <- Sys.time()
r_table <- arrow::read_feather("../data/processed/combined_data.feather")
print(class(r_table))
result <- r_table %>% count(model)
end_time <- Sys.time() 
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: 'dplyr'


R[write to console]: The following objects are masked from 'package:stats':

    filter, lag


R[write to console]: The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: 'arrow'


R[write to console]: The following object is masked from 'package:utils':

    timestamp




[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# ... with 17 more rows
Time difference of 37.20232 secs
Wall time: 39.8 s


## 6.2 `R` Observations

After a thorough discussion, our team have reached a **consensus that `Feather` file is the most suitable approach to transfer the dataframe from `Python` to `R`**. 

> We used the **speed** of read / write and the **ability to support various operations** as the **criteria** to make the decision.

* As the input data is around 5.7 GB, it is extremely slow to use `Pandas` exchange, hence its rejection. 

* We do not select `Arrow` exchange as it only supports some operations ([link in Gittu's lecture note](https://arrow.apache.org/docs/r/articles/dataset.html)). 

* `Parquet` file, even though is quite fast, is still slower than `Feather` V2 version when being read. 

* Similarly, it is much faster to write into a `Feather` file than a `Parquet` file from a `Python` dataframe ([link in Gittu's lecture note](https://ursalabs.org/blog/2020-feather-v2/)). 

* Even though a `Parquet` file can save more storage space, it is not a main concern for us as storage cost is cheap given this file size.